In [1]:
import json

In [3]:
comment_json = {
                    'id': "comment.id",
                    'name': "comment.name",
                    'author': "comment.author.name",
                    'body': "comment.body",
                }

In [17]:
data = json.dumps(comment_json)

In [18]:
print(data)

{"id": "comment.id", "name": "comment.name", "author": "comment.author.name", "body": "comment.body"}


In [13]:
dt = json.loads(data)

In [19]:
test_comment  = {'id': 'lhsyiws', 'name': 't1_lhsyiws', 'author': 'goodyear_1678', 'body': 'Hmmm, turns out your lot are a bit too small as well apparently!', 'upvotes': 1, 'downvotes': 0, 'timestamp': 1723495017.0, 'match_keywords': '1', 'sentiment_id': 'Positive'}

In [20]:
print(json.dumps(test_comment))

{"id": "lhsyiws", "name": "t1_lhsyiws", "author": "goodyear_1678", "body": "Hmmm, turns out your lot are a bit too small as well apparently!", "upvotes": 1, "downvotes": 0, "timestamp": 1723495017.0, "match_keywords": "1", "sentiment_id": "Positive"}
